# 02_baseline_tfidf — Binario A/D

**Objetivo:** baseline clásico **robusto a ruido** (typos/transcripción) con **char TF‑IDF (3–5)** + **SVM (LinearSVC)**.  
**Justificación:** los n‑gramas de caracteres capturan patrones ortográficos aun con errores; es un buen contrapunto al enfoque rule‑based.


In [1]:
# === Paths / Globals (auto-detect) ===
from pathlib import Path
import pandas as pd
import re, unicodedata, os

# Rutas y entorno
BASE_PATH = Path.cwd()
if BASE_PATH.name == "notebooks":
    BASE_PATH = BASE_PATH.parent

DATA_PATH = BASE_PATH / "data"
FORK_PATH = BASE_PATH / "Spanish_Psych_Phenotyping_PY"

# Reuse existing globals if present (from your 02_baselines.ipynb)
DATA_PATH = Path(DATA_PATH) if 'DATA_PATH' in globals() else Path('data')
FORK_PATH = Path(FORK_PATH) if 'FORK_PATH' in globals() else Path('Spanish_Psych_Phenotyping_PY')
DATA_PATH.mkdir(exist_ok=True)


INPUT_FILE   = DATA_PATH/'ips_raw.csv'
if not INPUT_FILE.exists():
    raise FileNotFoundError("No se encontró ni ips_clean.csv ni ips_raw.csv en " + str(DATA_PATH))

print("📥 INPUT_FILE:", INPUT_FILE)

# --- Columnas reales de tu dataset ---
TEXT_COL = "texto"
LABEL_COL = "etiqueta"

# Column preferences (honor globals if defined)
TEXT_COL  = TEXT_COL  if 'TEXT_COL'  in globals() else None
LABEL_COL = LABEL_COL if 'LABEL_COL' in globals() else None

def _guess_text_col(df):
    if TEXT_COL and TEXT_COL in df.columns: 
        return TEXT_COL
    for c in ['texto','Motivo Consulta','original_motivo_consulta','text']:
        if c in df.columns: return c
    for c in df.columns:
        if df[c].dtype == 'O': return c
    raise ValueError("No se encontró columna de texto.")

def _guess_label_col(df):
    if LABEL_COL and LABEL_COL in df.columns: 
        return LABEL_COL
    for c in ['etiqueta','Tipo','label','target','y','clase']:
        if c in df.columns: return c
    return None

def _norm_label_bin(s):
    if pd.isna(s): return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    return {'depresivo':'depresion'}.get(s, s)


📥 INPUT_FILE: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/ips_raw.csv


## 1) Carga y preprocesamiento **agresivo** (pensado para ML clásico)

In [2]:
import pandas as pd, re, unicodedata

# === CARGAR SPLITS UNIFICADOS ===
SPLITS_PATH = DATA_PATH / "splits"
dataset_base = pd.read_csv(SPLITS_PATH / 'dataset_base.csv')
train_indices = pd.read_csv(SPLITS_PATH / 'train_indices.csv')['row_id'].values
val_indices = pd.read_csv(SPLITS_PATH / 'val_indices.csv')['row_id'].values

print(f"✅ Splits cargados: Train={len(train_indices)} | Val={len(val_indices)}")

text_col  = _guess_text_col(dataset_base)
label_col = _guess_label_col(dataset_base)

# TF-IDF: agresivo (lower, símbolos, alargamientos, proxy negación 'no_X')
RE_MULTI = re.compile(r'(.)\1{2,}')
def clean_text_ml(s: str) -> str:
    if pd.isna(s): return ""
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFC", s)
    s = RE_MULTI.sub(r'\1\1', s)
    s = re.sub(r"[^a-z0-9áéíóúüñ\s.,!?:/\-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    s = re.sub(r"\bno\s+([a-záéíóúüñ]{2,})", r"no_\1", s)  # negación simple
    return s

dataset_base['texto_ml'] = dataset_base[text_col].map(clean_text_ml)

# Separar train y val usando índices guardados
df_train = dataset_base[dataset_base['row_id'].isin(train_indices)].copy()
df_val = dataset_base[dataset_base['row_id'].isin(val_indices)].copy()

X_train, y_train = df_train['texto_ml'], df_train[label_col]
X_val, y_val = df_val['texto_ml'], df_val[label_col]

print(f"Train distribución:\n{y_train.value_counts()}")
print(f"Val distribución:\n{y_val.value_counts()}")

✅ Splits cargados: Train=2518 | Val=630
Train distribución:
Tipo
depresion    1778
ansiedad      740
Name: count, dtype: int64
Val distribución:
Tipo
depresion    445
ansiedad     185
Name: count, dtype: int64
Train distribución:
Tipo
depresion    1778
ansiedad      740
Name: count, dtype: int64
Val distribución:
Tipo
depresion    445
ansiedad     185
Name: count, dtype: int64


## 2) Split estratificado y entrenamiento (char TF‑IDF + LinearSVC)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

tfidf_char = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(3,5),
    min_df=2,
    max_df=0.95
)

clf = Pipeline([
    ('tfidf', tfidf_char),
    ('svm', LinearSVC(class_weight='balanced', random_state=42))
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print("✅ Entrenamiento completado.")

✅ Entrenamiento completado.


## 3) Métricas y exportables

In [4]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

tfidf_pred_csv   = DATA_PATH/'tfidf_predictions.csv'
tfidf_report_csv = DATA_PATH/'tfidf_classification_report.csv'
tfidf_eval_csv   = DATA_PATH/'tfidf_eval.csv'
tfidf_cm_csv     = DATA_PATH/'tfidf_confusion_matrix.csv'

classes = ['depresion','ansiedad']

pd.DataFrame(classification_report(y_val, y_pred, labels=classes, output_dict=True, zero_division=0))  .transpose().to_csv(tfidf_report_csv, index=True, encoding='utf-8')

pd.DataFrame([{
    'macro_f1': f1_score(y_val, y_pred, average='macro', zero_division=0),
    'macro_precision': precision_score(y_val, y_pred, average='macro', zero_division=0),
    'macro_recall': recall_score(y_val, y_pred, average='macro', zero_division=0),
    'n': int(len(y_val))
}]).to_csv(tfidf_eval_csv, index=False, encoding='utf-8')

cm = confusion_matrix(y_val, y_pred, labels=classes)
pd.DataFrame(cm, index=[f'true_{c}' for c in classes], columns=[f'pred_{c}' for c in classes]).to_csv(tfidf_cm_csv)

pd.DataFrame({'texto': X_val, 'y_true': y_val, 'y_pred': y_pred}).to_csv(tfidf_pred_csv, index=False, encoding='utf-8')

print("✅ Exportados:")
print(" - Predicciones:", tfidf_pred_csv)
print(" - Reporte:", tfidf_report_csv)
print(" - Métricas:", tfidf_eval_csv)
print(" - Matriz:", tfidf_cm_csv)

✅ Exportados:
 - Predicciones: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_predictions.csv
 - Reporte: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_classification_report.csv
 - Métricas: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_eval.csv
 - Matriz: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_confusion_matrix.csv
